## Evaluating and Analyzing of XGBoost Trainined Models for Detecting Adverse Events

In this notebook, we are going to analyze a trained xgboost models to detect adverse events for given patients based on their historical data.
Let's first start to import the packages needed.

In [2]:
import json
import os
import numpy as np
import pandas as pd
import shutil

from urllib.parse import urlparse

import boto3

import shap
import tarfile
import pickle

import matplotlib.pyplot as plt
#%matplotlib inline

pd.options.mode.chained_assignment = None

import xgboost as xgb

##User defined import
from metrics import compute_metrics

In [3]:
def copy_model_from_s3(s3_model_path, local_model_dir):
    """Copy model from s3 to local
    Args:
        s3_model_path(str): S3 path where the model gz is saved
    Returns:
        Destination model path
    """
    client = boto3.client('s3')
    o = urlparse(s3_model_path)
    bucket = o.netloc
    key = o.path
    key = key.lstrip('/')
    if not os.path.exists(local_model_dir): 
        os.makedirs(local_model_dir) 
    fname = os.path.basename(s3_model_path) 
    output_path = os.path.join(local_model_dir, fname)
    
    client.download_file(bucket, key, output_path)
    
    return output_path
   

def load_model(gz_model_path): 
    """
    Loads xgboost trained model from disk
    Args:
        gz_model_path(str): Compressed Model path
    Returns:
        xgboost: Xgboost model object
    """
    model_dir = os.path.dirname(gz_model_path)
    model_path = os.path.join(model_dir, 'xgboost-model')

    tar = tarfile.open(gz_model_path, "r:gz")
    tar.extractall(model_dir)
    tar.close()
    
    #Load Model
    model = pickle.load(open(model_path, "rb"))
    
    #Remove the local copy of the model files
    shutil.rmtree(model_dir)

    return model


def get_labels_scores(df_preds_labels, target_names=None):
    """Get labels and scores/predictions to compute model metrics
    Args:
        df_preds_labels(pd.DataFrame): Dataframe of predictions & true labels
        target_names(list): List of target events
    Returns:
        Tuple of labels(np.array), scores(np.array) and Event names(list)
    """
    labels = None
    scores = None
    if target_names is None:
        cols = df_preds_labels.columns.tolist()
        label_names = [col for col in cols if not col.endswith('_')]
        label_names = [name for name in label_names if not name.endswith('probs')]
        pred_names = [col for col in cols if col.endswith('probs')]
    else:
        label_names = target_names
        pred_names = [name+'_probs' for name in target_names]
    
    labels = df_preds_labels[label_names].values
    scores = df_preds_labels[pred_names].values

    return labels, scores, label_names


In [27]:
PREPROCESSED_DATA_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/ae/1000/preprocessed'
SPLIT = 'test'
NUM_FEATURES = 200
DATA_PATH = os.path.join(PREPROCESSED_DATA_DIR, SPLIT+'.csv')

TRAIN_DATA_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/ae/1000/training/'
MODEL_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/ae/1000/model/'

TRAIN_RESULTS_PATH = os.path.join(TRAIN_DATA_DIR, 'train_results', str(NUM_FEATURES), 'train_results.csv')
FINAL_RESULTS_DIR = os.path.join(TRAIN_DATA_DIR, 'train_results', str(NUM_FEATURES), 'final_results')

Now, we will add all the values/paths needed to train the models

In [28]:
df_results = pd.read_csv(TRAIN_RESULTS_PATH)
df_results.head()

,class,num_features,val_auc,best_model_path
0,d_5990,200,0.8148,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
1,d_78605,200,0.8208,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
2,d_486,200,0.8714,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
3,d_78650,200,0.7411,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
4,d_78079,200,0.7501,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...


In [29]:
# df_vis = df_results.pivot(index='num_features', columns='class', values='val_auc')
# df_vis.plot()

In [30]:
#Get models having the best performance for each target variable
idx = df_results.groupby('class')['val_auc'].transform(max) ==df_results['val_auc']
df_best = df_results[idx]
print(df_best.shape)
df_best.head()

(20, 4)


,class,num_features,val_auc,best_model_path
0,d_5990,200,0.8148,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
1,d_78605,200,0.8208,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
2,d_486,200,0.8714,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
3,d_78650,200,0.7411,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...
4,d_78079,200,0.7501,s3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/...


In [31]:
# best_models = [['d_5990', 100, 0.7, 's3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/ae/final/month-0/xgboost/2020-11-10-20-48-57/100/d_5990/output/sagemaker-xgboost-201110-2049-020-212dc74f/output/model.tar.gz'],
#                ['d_78605', 100, 0.6, 's3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/ae/final/month-0/xgboost/2020-11-10-20-48-57/100/d_5990/output/sagemaker-xgboost-201110-2049-016-3e3ab8f4/output/model.tar.gz']]
# columns = ['class', 'num_features', 'val_auc', 'best_model_path']
# df_best = pd.DataFrame(best_models, columns=columns)
# print(df_best.shape)
# df_best.head()

In [32]:
df_data = pd.read_csv(DATA_PATH)
print(df_data.shape)
df_data.head()

(4485796, 320)


,h_99213,h_99214,h_36415,d_25000,p_D1E,d_4019,h_85025,h_80053,h_97110,d_4011,...,d_5789,d_78791,d_6826,d_78659,d_78907,d_7840,d_28860,d_4660,d_6829,d_00845
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
def get_model_predictions(row, df_data, local_model_dir):
    """Process the predictions and performance for best model for each class.
    df_data first column is labels and others are features
    """
    best_model_path = row['best_model_path']
    target = row['class']
    num_features = row['num_features']
    
    #Copy the best model from s3 to local
    output_path = copy_model_from_s3(best_model_path, local_model_dir)

    #Load the copied model
    model = load_model(output_path)
    
    preds = []
    features = df_data.columns.tolist()[:num_features]
    #Predict for data and save in pd Dataframe
    probs = model.predict(xgb.DMatrix(df_data[features].values, df_data[target].values))
    #probs = model.predict(xgb.DMatrix(df_data.iloc[:, :num_features], df_data[target].values, feature_names=feature_names))
    preds.append(df_data[target].tolist())
    preds.append((probs>=0.5).astype(int).tolist())
    preds.append(probs.tolist())
    
    columns = [target, target+'_', target+'_probs']
    return preds, columns


def get_all_predictions(df_best_models, df_data, local_model_dir):
    """Get predictions from each of the best models of each target variable."""
    num_rows = df_best_models.shape[0]
    all_columns = []
    all_preds = []
    for i in range(num_rows):
        row = df_best_models.iloc[i, :]
        preds, columns = get_model_predictions(row, df_data, local_model_dir)
        all_preds += preds
        all_columns += columns
        
    df_preds = pd.DataFrame(np.array(all_preds).T, columns=all_columns)
    return df_preds

Evaluate for a sample model

In [34]:
# target = df_best.iloc[0, 0]
# num_features = df_best.iloc[0,1]
# best_model_path = df_best.iloc[0, 3]

# #Copy the best model from s3 to local
# output_path = copy_model_from_s3(best_model_path, MODEL_DIR)
# #Load the copied model
# model = load_model(output_path)
# #model.feature_names

# #Evaluate model on data
# feature_names = df_data.columns.tolist()[:num_features]
# auc = model.eval(xgb.DMatrix(df_data[feature_names].values, df_data[target].values))
# print('AUC: - {}'.format(auc))

In [35]:
df_preds = get_all_predictions(df_best, df_data, MODEL_DIR)

In [36]:
print(df_preds.shape)
df_preds.head()

(4485796, 60)


,d_5990,d_5990_,d_5990_probs,d_78605,d_78605_,d_78605_probs,d_486,d_486_,d_486_probs,d_78650,...,d_28860_probs,d_4660,d_4660_,d_4660_probs,d_6829,d_6829_,d_6829_probs,d_00845,d_00845_,d_00845_probs
0,0.0,0.0,0.417892,0.0,0.0,0.266474,0.0,0.0,0.212412,0.0,...,0.306573,0.0,0.0,0.471074,0.0,1.0,0.501722,0.0,0.0,0.190965
1,0.0,1.0,0.828509,0.0,0.0,0.369781,0.0,0.0,0.267425,0.0,...,0.325719,0.0,0.0,0.366931,0.0,0.0,0.331917,0.0,0.0,0.284323
2,0.0,1.0,0.570034,0.0,1.0,0.559346,0.0,0.0,0.357956,0.0,...,0.375516,0.0,0.0,0.492580,0.0,1.0,0.575691,0.0,0.0,0.323652
3,0.0,1.0,0.563573,0.0,0.0,0.372952,0.0,0.0,0.465404,0.0,...,0.418124,0.0,1.0,0.533306,0.0,1.0,0.662477,0.0,0.0,0.423647
4,0.0,1.0,0.525949,0.0,1.0,0.779116,0.0,1.0,0.779511,0.0,...,0.528974,0.0,0.0,0.323173,0.0,1.0,0.606252,0.0,1.0,0.741166


In [37]:
np_labels, np_scores, _ = get_labels_scores(df_preds)
target_names = df_best['class'].tolist()
df_metrics = compute_metrics(np_labels, np_scores, target_names=target_names)

In [38]:
print('Labels Shape: {}, Scores Shape: {}'.format(np_labels.shape, np_scores.shape))
df_metrics.head()

Labels Shape: (4485796, 20), Scores Shape: (4485796, 20)


,auroc,avgpr,precis_5%,recall_5%,precis_2%,recall_2%,precis_1%,recall_1%,precis_0.5%,recall_0.5%,precis_0.25%,recall_0.25%,calib_mean,calib_mse
d_5990,0.778992,0.109322,0.141099,0.313666,0.184114,0.163716,0.210041,0.093385,0.233537,0.051916,0.250736,0.027871,0.380746,0.191113
d_78605,0.783012,0.106690,0.134018,0.303280,0.180046,0.162976,0.212783,0.096304,0.242186,0.054806,0.266072,0.030107,0.367241,0.182405
d_486,0.840960,0.096153,0.083316,0.438466,0.126935,0.267205,0.165545,0.174242,0.206964,0.108919,0.251092,0.066074,0.309230,0.138763
d_78650,0.715866,0.065416,0.086237,0.216873,0.116579,0.117272,0.141937,0.071390,0.171385,0.043101,0.200535,0.025217,0.427653,0.219333
d_78079,0.694288,0.059651,0.087494,0.181149,0.104140,0.086245,0.114584,0.047447,0.123322,0.025533,0.132679,0.013736,0.440258,0.233167


In [39]:
#pd.DataFrame(df_metrics.mean()).T

In [40]:
mn = df_metrics.min()
mx = df_metrics.max()
avg = df_metrics.mean()

df_metrics.loc['Min'] = mn
df_metrics.loc['Max'] = mx
df_metrics.loc['Average'] = avg

In [41]:
df_metrics.tail()

,auroc,avgpr,precis_5%,recall_5%,precis_2%,recall_2%,precis_1%,recall_1%,precis_0.5%,recall_0.5%,precis_0.25%,recall_0.25%,calib_mean,calib_mse
d_6829,0.730072,0.010547,0.013478,0.258177,0.019261,0.147579,0.023787,0.091126,0.028356,0.054317,0.030406,0.029123,0.445499,0.219398
d_00845,0.899362,0.018270,0.011570,0.626358,0.019305,0.418055,0.026127,0.282887,0.031745,0.171856,0.037628,0.101859,0.254197,0.106176
Min,0.674839,0.010547,0.011570,0.173897,0.019261,0.086245,0.023787,0.047447,0.028356,0.025533,0.030406,0.013736,0.247464,0.106176
Max,0.899362,0.201784,0.242689,0.626358,0.251979,0.418055,0.255100,0.282887,0.260154,0.171856,0.266072,0.101859,0.460495,0.233970
Average,0.760703,0.049622,0.060146,0.298890,0.077531,0.167899,0.090915,0.103997,0.104195,0.061832,0.116888,0.035822,0.393058,0.193265


In [42]:
feature_names = df_data.columns.tolist()[:NUM_FEATURES]
if not os.path.exists(FINAL_RESULTS_DIR):
    os.makedirs(FINAL_RESULTS_DIR)
    
#Save the features used
features_list_path = os.path.join(FINAL_RESULTS_DIR, 'features.txt')
with open(features_list_path, 'w') as fp:
    fp.write('\n'.join(feature_names))

#Save the final metrics results
final_results_path = os.path.join(FINAL_RESULTS_DIR, SPLIT+'_metrics.csv')
df_metrics.to_csv(final_results_path)

## Explainability and Visualization using SHAP (SHapley Additive exPlanations)

*Source: https://github.com/slundberg/shap*

In [10]:
import warnings
warnings.filterwarnings("ignore")

import shap
import matplotlib.pyplot as plt
%matplotlib inline

# load JS visualization code to notebook
#shap.initjs()

In [ ]:
print('Processing for {} data...'.format(SPLIT))
feature_names = df_data.columns.tolist()[:NUM_FEATURES]
X = df_data[feature_names]

#Create a new shap dir if not available
shap_dir = os.path.join(FINAL_RESULTS_DIR, 'shap_'+SPLIT)
if not os.path.exists(shap_dir):
    os.makedirs(shap_dir)
    
num_rows = df_best.shape[0]
for i in range(num_rows):
    target = df_best.iloc[i, 0]
    num_features = df_best.iloc[i, 1]
    best_model_path = df_best.iloc[i, 3]

    y = df_data[target]

    #Copy the best model from s3 to local
    output_path = copy_model_from_s3(best_model_path, MODEL_DIR)
    #Load the copied model
    model = load_model(output_path)
    
    # explain the model's predictions using SHAP
    # (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    print('Computing SHAP Results for Target={}...'.format(target))
    
#     vis_path = os.path.join(shap_dir, target+'_shap_values.pkl')
#     with open(vis_path, 'wb') as fp:
#         pickle.dump(shap_values, fp)
        
    # visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
    vis_path = os.path.join(shap_dir, target+'_per_patient_shap.png')
    shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:], matplotlib=True, show=False)
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
    # visualize the training set predictions
    #shap.force_plot(explainer.expected_value, shap_values, X) ## Out-of-memory Error
    
    # create a dependence plot to show the effect of a single feature across the whole dataset
    vis_path = os.path.join(shap_dir, target+'_per_feature_shap.png')
    shap.dependence_plot(feature_names[0], shap_values, X, show=False)
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
    # summarize the effects of all the features
    shap.summary_plot(shap_values, X, show=False)
    vis_path = os.path.join(shap_dir, target+'_all_features_shap.png')
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
    #Compute the mean absolute value of the SHAP values for each feature to get a standard bar plot
    shap.summary_plot(shap_values, X, plot_type="bar", show=False)
    vis_path = os.path.join(shap_dir, target+'_all_features_importance.png')
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
print('Shap Values and Visualizations Successfully Saved to {}!'.format(shap_dir))

Processing for val data...
Computing SHAP Results for Target=d_5990...
Computing SHAP Results for Target=d_78605...
Computing SHAP Results for Target=d_486...
Computing SHAP Results for Target=d_78650...
